# Simplified Data Encryption Standard (DES)

### by: Beatriz Vela 

## What is DES? 

Data Encryption Standard (DES) is the most widely used encryption algorithm in the world. It is a block cipher (operates on blocks 16 hexadecimal numbers long or 64 bits), and it is symmetric (meaning that it uses the same keys for encryption and decryption). 

It is based on the Feistel Cipher, and also operates on left blocks ($L_n$) and right blocks ($R_n$). However, some of the differences between Feistel and DES are that DES uses 16 rounds of the Feistel Structure using a transformation of the original key in each round (subkeys), the plain text also undergoes an initial permutation, and in each round a new function (analagous to the Feistel Function) is created by a series of permutations and XOR operations with the subkey before being XOR one last time with the left block of the previous round: $L_{n-1}$ 

For this Assignment, the Example by J. Orlin Grabbe provided by Dr. Lei in the resources was used. 

# Inputs 

Before explaining the code in this assignment, we'll start by asking an input for the key and plain text from the user which will be stored in `M` and `key_input`.

In [1]:
key_input = input("Enter the hexadecimal key: ")

M_input = input("Write the text you want to encrypt: ")

Enter the hexadecimal key: 133457799BBCDFF1
Write the text you want to encrypt: 0123456789ABCDEF


# Arrays Needed for Operations

As the DES algorithm operates on permutating bits - that is, changing their positions with respect to each other - the following arrays were created to facilitate the bit rearrangement process.

### IP table for Plain Text Permutation

This is the initial permutation of the message data $M$. Once the plain message is converted to binary, the position of the bits will be rearrenged using the guidance from the table. 

For example, as the table starts with positions 58, 50, and 42, this means that from the initial message $M$ the 58th bit will now be the 1st bit of the permutated message, the 50th bit of $M$ will be the 2nd, 42th bit of $M$ will be the third... until all the bits of $M$ have been rearranged. 

The array created for the IP permutation was named as `ip_table`

In [2]:
ip_table = [58, 50, 42, 34, 26, 18, 19, 2, 
            60, 52, 44, 36, 26, 28, 12, 4,
            62, 54, 46, 38, 30, 22, 14, 6,
            64, 56, 48, 40, 32, 24, 16, 8,
            57, 49, 41, 33, 25, 17, 9, 1,
            59, 51, 43, 35, 27, 19, 11, 3,
            61, 53, 45, 37, 29, 21, 13, 5,
            63, 55, 47, 39, 31, 23, 15, 7]

## Arrays for Subkey Generation

### PC-1 

PC-1 is the first permutation needed for the generation of the subkeys. It will take the bits from the original key $K$ and rearrange them. However, the original key $K$ starts from 64-bits, and after the permutation using `pc1_table`, only 56-bits of the original key will be left ($K_+$).

In [3]:
pc1_table = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

### Left Shift Table

After the first permutation using `pc1_table`, the permutated key ($K_+$) will be separated into two halves ($C_0$ and $D_0$) of 28 bits. These halves will be used for the generation of 16 blocks ($C_n$ and $D_n$). In which the bits used for the previous pair of $C_n-1$ and $D_n-1$ will be the reference to create the new pair of $C_n$ and $D_n$. The `shift_table` array will be used for guidance on the number of left shifts needing to happen to create blocks $C_n$ and $D_n$ for iterations ($n$) $1$ through $16$.

In [4]:
shift_table = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

### PC-2 Table 

`pc2_table` will be used to generate the 16 subkeys ($K_n$). Each pair of $C_n$ and $D_n$ which are 56 bits total will be permutated to be left with only 48 bits.

This will facilitate the first XOR operation needed for encryption later on. 

In [5]:
pc2_table = [14, 17, 11, 24, 1, 5,
             3, 28, 15, 6, 21, 10,
             23, 19, 12, 4, 26, 8,
             16, 7, 27, 20, 13, 2,
             41, 52, 31, 37, 47, 55,
             30, 40, 51, 45, 33, 48,
             44, 49, 39, 56, 34, 53,
             46, 42, 50, 36, 29, 32]

## Arrays for Function Generation in Each Round

To start with encryption, the plain text message is separated into two blocks ($L_0$ and $R_0$). 16 rounds will be completed as follows 
$$
L_n = R_{n-1}
R_n = L_{n-1}\oplus f(R_{n-1}, K_n)
$$

To generate $f(R_{n-1}, K_n)$ a series of transformations need to occur. 

### E Bit - Selection Table 

The first is a permutation using the right half block of the previous round ($R_{n-1}$) and `ebit_table`. This will rearrange the bits according to the positions in the table, and expand the right block from 32 bits to 48 bits. 

Again, this will itirate through sixteen rounds. For notation purposes, the outputs of this table will be referred to as $E(R_{n-1})$.

In [6]:
ebit_table = [32, 1, 2, 3, 4, 5, 4, 5,
              6, 7, 8, 9, 8, 9, 10, 11,
              12, 13, 12, 13, 14, 15, 16, 17,
              16, 17, 18, 19, 20, 21, 20, 21,
              22, 23, 24, 25, 24, 25, 26, 27,
              28, 29, 28, 29, 30, 31, 32, 1]

In each round, after the bits for each right block are expanded. $E(R_{n-1})$ will be XOR with each $K_n$ obtained from the permutation done by `pc2_table`. 

For each round, this will yield 48 bits, or 8 groups ($B_n$) of 6 bits. 
$$
K_n \oplus E(R_{n-1}) = B_1B_2B_3B_4B_5B_6B_7B_8
$$

### S-Box Tables (1-8)
Each $B_n$ will be "isolated" and cross-referenced to the S-Box table as follows: 

Each $B_n$ is composed of 6 bits. Putting together the 1st and 6th bits of $B_n$, represent a binary number in the decimal range of 0-3. That number will determine the "row" in the S table. The 2nd, 3rd, 4th, and 5th numbers together, represent a number in the decimal range 0-15 which will correspond to the "columns". The array for the `sbox_table` will be used to get the number located in the row and column determined for each $B_n$. This will later be translated into binary. 

Ideally, after running each $B_n$ though the `sbox_table`, we will have a total of 32 bits. 

In [25]:
#S-Box tables 1-8 
sbox_table = [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
         [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
         [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
         [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
 
        [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
         [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
         [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
         [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
 
        [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
         [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
         [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
         [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
 
        [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
         [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
         [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
         [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
 
        [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
         [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
         [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
         [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],
 
        [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
         [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
         [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
         [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],
 
        [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
         [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
         [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
         [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],
 
        [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
         [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
         [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
         [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]]

### P Table

The final step for creating $f(R_{n-1}, K_n)$ for each round is permutating the bits from each `sbox_table` output using `p_table`.

In other words, `p_table` will rearrange the bits one last time by taking a 32 bit input and generating a 32 bit output which will be $f(R_{n-1}, K_n)$. for each round. 

Then, $f(R_{n-1}, K_n)$ can be XOR with $L_{n-1}$ to get the right half block for the round as given by: 
$$
R_n = L_{n-1}\oplus f(R_{n-1}, K_n)
$$

In [8]:
p_table = [16,  7, 20, 21,
       29, 12, 28, 17,
       1, 15, 23, 26,
       5, 18, 31, 10,
       2,  8, 24, 14,
       32, 27,  3,  9,
       19, 13, 30,  6,
       22, 11,  4, 25]

## Final Permutation

This permutation happens in the 16th round. When we are at $L_16$ and $R_16$, a swap will be made ($R_16L_16$) and the bits rearranged using `ipfinal_table`.

In [9]:
ipfinal_table = [40, 8, 48, 16, 56, 24, 64, 32,
                 39, 7, 47, 15, 55, 23, 63, 31,
                 38, 6, 46, 14, 54, 22, 62, 30,
                 37, 5, 45, 13, 53, 21, 61, 29,
                 36, 4, 44, 12, 52, 20, 60, 28,
                 35, 3, 43, 11, 51, 19, 59, 27,
                 34, 2, 42, 10, 50, 18, 58, 26,
                 33, 1, 41, 9, 49, 17, 57, 25]

# Defining Functions 

DES contains a lot of operations that are repetitive (happen throughout 16 rounds). As a strategy, the following functions were defined to facilitate the operations.

The first function `hextobin` will be used to convert hexadecimal values (inputed by user) into binary. 

A dictionary was defined to facilitate the mapping between hexadecimal (0123456789ABCDEF) and their binary representation as other attempts conversion attempts were ommitting leading zeros which heavily impacted an accurate encryption. 

In [10]:
def hextobin(hexa):
    h2b_dic = {'0': "0000",
          '1': "0001",
          '2': "0010",
          '3': "0011",
          '4': "0100",
          '5': "0101",
          '6': "0110",
          '7': "0111",
          '8': "1000",
          '9': "1001",
          'A': "1010",
          'a': "1010",
          'B': "1011",
          'b': "1011",
          'C': "1100",
          'c': "1100",
          'D': "1101",
          'd': "1101",
          'E': "1110",
          'e': "1110",
          'F': "1111",
          'f': "1111"}
    conv = ""
    for i in range(len(hexa)):
        conv = conv + h2b_dic[hexa[i]]
    return conv    

## First code attempeted that didn't work though iterations as it wasn't 
#displaying leading zeros

#def hextobin(hexa):
    #conv = "{0:08b}".format(int(hexa, 16))
    #return conv

`bintohex` will be used to convert binary into its hexadecimal representation. 

A dictionary was defined again to facilitate the mapping between hexadecimal (0123456789ABCDEF) and their binary representation. As previous versions of the code traced errors back when doing round iterations.

In [11]:
def bintohex(binary):
    b2h_dic = {"0000": '0',
          "0001": '1',
          "0010": '2',
          "0011": '3',
          "0100": '4',
          "0101": '5',
          "0110": '6',
          "0111": '7',
          "1000": '8',
          "1001": '9',
          "1010": 'A',
          "1011": 'B',
          "1100": 'C',
          "1101": 'D',
          "1110": 'E',
          "1111": 'F',
          "1010": 'a',
          "1011": 'b',
          "1100": 'c',
          "1101": 'd',
          "1110": 'e',
          "1111": 'f'}
    conv = ""
    for i in range(0, len(binary), 4):
        ch = ""
        ch = ch + binary[i:i + 4]
        conv = conv + b2h_dic[ch] 
    return conv


## First code attempted showed an error when itierating through the 16 rounds:
#num = int(binary, 2)
    #conv = format(num, 'x')
    #return conv

`bintodec` will be used to convert binary numbers into their decimal representation.

In [12]:
def bintodec(binary):
    #b = binary
    num, i, n = 0, 0, 0
    while(binary != 0):
        dec = binary % 10
        num = num + dec * pow(2, i)
        binary = binary//10
        i += 1
    return num

## First code attempted (failed through iterations)

#def bintodec(binary):
    #conv = int(binary,2)
    #return conv

To be used to convert the output of the S-boxes (decimal) into machine language. 

`dectobin` converts a decimal numcer into machine language making sure it ends up with a string of binary numbers

In [13]:
def dectobin(num):
    conv = bin(num).replace("0b", "")
    if(len(conv)%4 != 0):
        div = len(conv) / 4
        div = int(div)
        counter = (4 * (div + 1)) - len(conv)
        for i in range(0, counter):
            conv = '0' + conv        
    return conv

As permuation are one of the operations that happen the most during DES `permutation` helps iterate through the bit rearrangements. 

In [14]:
def permutation(bstr, table, n): #function of binary string, array, and size n
    perm = ""
    for i in range(0, n): #limiting according to number in the array
        perm = perm + bstr[table[i] - 1] #adding on to the empty string 
    return perm

`left_shift` was defined to help read through the `shift_table`

In [15]:
def left_shift(bstr, n): #input binary str and length table
    stg = "" 
    for i in range(n): 
        for j in range (1, len(bstr)): #to iterate only through the string
            stg = stg + bstr[j] #list bin
        stg = stg + bstr[0]
        bstr = stg
        stg = ""
    return bstr

`xor` was defined to do the XOR operations needed for each round

In [16]:
def xor(a,b):
    tmp = ""
    
    for i in range(len(a)):
        
        if (a[i]==b[i]): #checks that bin num are equal 
            tmp += "0" #adds and assigns 0 if true
            
        else: 
            tmp += "1" #if bin nums are not equal then they will be 1 
    return tmp

### Other functions defined but not used in this version:

Attempt to facilitate the shifts for $C_n$ and $D_n$ but found out it was better to code outside of a function

In [17]:
#def shifted(a,b):
    #for i in range(0, 16):
     #   a = left_shift(a, shift_table[i])
      #  b = left_shift(b, shift_table[i])
       # combine = a + b
        #return combine

The following two functions are in an attempt to take any plain text such as "Hello, World" and encrypt it using DES

In [18]:
def ptobin(pt):
    pt_ascii = [ord(x) for x in pt]
    pt_binary = [format(y, '08b') for y in pt_ascii]
    pt_binary = "".join(pt_binary)
    return pt_binary

In [19]:
def retrieveplain(binary):
    tmp = ''
    for i in range(0, len(binary),8):
        data = binary[i : i + 8]
        tmp = tmp + chr(bintodec(data))
    return tmp

# Key Generation

After defining the functions, now they are put to use during the Key Generation Algorithm: 

In [20]:
K = hextobin(key_input) #convert inputted hex key to bin. 


#First permutation with table PC-1
K_plus = permutation(K, pc1_table, 56)
print("\n The key after permutation with PC-1 table is: ", K_plus, 
      "\n The length of the key is: ", len(K_plus))

#Splitting into C0 and C0
C0 = K_plus[0:28] 
D0 = K_plus[28:56]

print("\n After splitting, C0 is: ", C0,
      "\n Length of C0 is: ", len(C0),
      "\n After splitting, D0 is: ", D0,
      "\n Length of D0 is: ", len(D0))

#Shift table 
Kn = [] #empty to stort the data for each round
Kn_hex = [] #hexadecimal conversion to display subkey for each round

for i in range(0, 16): #16 rounds
    C0 = left_shift(C0, shift_table[i]) #shifting bits using left_shift function 
    D0 = left_shift(D0, shift_table[i]) # and according to table
    CnDn = C0 + D0 # merging to prepare for subkey permutation for f(R,K)
    subkey = permutation(CnDn, pc2_table, 48) #permutation of merge, and PC-2
    print("\nRound: ", i+1,
          "\n Bits shifted: ", shift_table[i],
          "\n Cn is: ", C0, 
          "\n Dn is: ", D0,
          "\n After permutation with PC-2 table we have: ", subkey,
          "\n Subkey is: ", bintohex(subkey))
    
    Kn.append(subkey) #add item after loop  


    Kn_hex.append(bintohex(subkey)) 



#print(Kn.append(subkey))    

#print(Kn_hex[i])



 The key after permutation with PC-1 table is:  11110000110011001010101011110101010101100110011110001111 
 The length of the key is:  56

 After splitting, C0 is:  1111000011001100101010101111 
 Length of C0 is:  28 
 After splitting, D0 is:  0101010101100110011110001111 
 Length of D0 is:  28

Round:  1 
 Bits shifted:  1 
 Cn is:  1110000110011001010101011111 
 Dn is:  1010101011001100111100011110 
 After permutation with PC-2 table we have:  000110110000001011101111111111000111000001110010 
 Subkey is:  1b02effc7072

Round:  2 
 Bits shifted:  1 
 Cn is:  1100001100110010101010111111 
 Dn is:  0101010110011001111000111101 
 After permutation with PC-2 table we have:  011110011010111011011001110110111100100111100101 
 Subkey is:  79aed9dbc9e5

Round:  3 
 Bits shifted:  2 
 Cn is:  0000110011001010101011111111 
 Dn is:  0101011001100111100011110101 
 After permutation with PC-2 table we have:  010101011111110010001010010000101100111110011001 
 Subkey is:  55fc8a42cf99

Round:  4 
 B

# Encryption

In [42]:
def encryption(M, Kn, Kn_hex):
    M = hextobin(M_input)
    #Initial permutation of M using IP table
    M_bin = permutation(M, ip_table, 64)
    #print("\n Inputted message: ", M_input,
         # "\n Message translated to binary: ", M, 
         # "\n Message after permutation is: ", M_bin)
    #Separate into L0 and R0
    Ln = M_bin[0:32]
    Rn = M_bin[32:64]
    #print("\n Ln :", Ln,
     #     "\n Rn :", Rn,)
    
    #Calculating f(Rn, Kn)
    for i in range(0, 16):
        #Doing e-bit expansion 
        ebit_exp = permutation(Rn, ebit_table, 48)
        #E bit digits are XOR with round key Kn
        xor1 = xor(ebit_exp, Kn[i])
        print("Round: ", i+1, 
              "\n E-BIT Expansion", ebit_exp,
              "\n Subkey: ", Kn[i],
              "\n XOR with Kn: ", xor1)
        #Reading S-box table 
        SnBn = ""
        for j in range(0,8):
            #considering indexing starting from 0, row function gets the first
            #and last digits of every 6 block of bin numbers
            row = bintodec(int(xor1[j * 6] + xor1[(j * 6) + 5]))
            #gets numbers in between for the column  
            col = bintodec(
                int(xor1[(j * 6) + 1] + xor1[(j * 6) + 2] + xor1[(j * 6) + 3] + xor1[(j * 6 )+ 4]))
            #getting the location
            loc = sbox_table[j][row][col]
            #iteration and conversion from num to binary to get the final string
            SnBn = SnBn + dectobin(loc)
            print(i, SnBn)
        #Permutation with p_table for each SnBn to get f(Rn, Kn)
        SnBn1 = permutation(SnBn, p_table, 32)
     #   print("Calculated DES Function is: ", SnBn)
        
        #XOR with Ln
        print("\n",Ln, 
              "\n", SnBn1)
        Rn1 = xor(Ln, SnBn1)
        print("\n",Rn1)
        Ln = Rn1
      #  print ("Ln: ", Ln, 
       #     "\n DES Function XOR with Ln: ", Rn)
        #Swapping at final round 
        if (i != 15):
            Ln, Rn = Rn, Ln
        print ("Round ", i + 1, " ", bintohex(Ln), " ", bintohex(Rn), " ", "Round Key: ", Kn_hex[i])
        
    #Final combination 
    reverse = Ln + Rn 
        
    #Final permutation to get ciphertext
    C = permutation(reverse, ipfinal_table, 64)
    return C

print("Encryption")
C = encryption(M_input, Kn, Kn_hex)
C = bintohex(C)
print ("Cipher Text is: ", C)

Encryption
Round:  1 
 E-BIT Expansion 011110100001010101010101011110100001010101010101 
 Subkey:  000110110000001011101111111111000111000001110010 
 XOR with Kn:  011000010001011110111010100001100110010100100111
0 0101
0 01011100
0 010111001000
0 0101110010000010
0 01011100100000101011
0 010111001000001010110101
0 0101110010000010101101011001
0 01011100100000101011010110010111

 11001100000010001100110011111111 
 00100011010010101010100110111011

 11101111010000100110010101000100
Round  1   f0aaf0aa   ef426544   Round Key:  1b02effc7072
Round:  2 
 E-BIT Expansion 011101011110101000000100001100001010101000001001 
 Subkey:  011110011010111011011001110110111100100111100101 
 XOR with Kn:  000011000100010011011101111010110110001111101100
1 1111
1 11111000
1 111110001000
1 1111100010001110
1 11111000100011100011
1 111110001000111000111010
1 1111100010001110001110101010
1 11111000100011100011101010101110

 11110000101010101111000010101010 
 00111000111010101001011111100011

 11001000010000

# Decryption

In [22]:
print("Decryption")
Kn_rev = Kn[::-1]
Kn_hex_rev = Kn_hex[::-1]
text = bintohex(encryption(C, Kn_rev, Kn_hex_rev))
print("Plain Text : ", text)

Decryption
Round:  1 
 E-BIT Expansion 011110100001010101010101011110100001010101010101 
 Subkey:  110010110011110110001011000011100001011111110101 
 XOR with Kn:  101100010010100011011110011101000000001010100000
0 0010
0 00100111
0 001001111010
0 0010011110101111
0 00100111101011111000
0 001001111010111110001100
0 0010011110101111100011000000
0 00100111101011111000110000000111
Round  1   f0aaf0aa   1d4a9783   Round Key:  cb3d8b0e17f5
Round:  2 
 E-BIT Expansion 100011111010101001010101010010101111110000000110 
 Subkey:  101111111001000110001101001111010011111100001010 
 XOR with Kn:  001100000011101111011000011101111100001100001100
1 1011
1 10111101
1 101111010111
1 1011110101111011
1 10111101011110111000
1 101111010111101110001011
1 1011110101111011100010111000
1 10111101011110111000101110001011
Round  2   1d4a9783   6d419afd   Round Key:  bf918d3d3f0a
Round:  3 
 E-BIT Expansion 101101011010101000000011110011110101011111111010 
 Subkey:  010111110100001110110111111100101110011100111

## Conclusion and Future Developments

A lot of this code was trial and error. It was found that a lot of the predefined functions had to changed once they were processed by the Key Generation and Encryption algorithms. This was often due to the output generated by the functions during the development of this code sometimes was not a complete binary string. 

As seen in the error above, the state of this code yields progress with the first round of encryption. However, it is when iterating through the other rounds where there is a failure which is believeed to be traced back on the portion of the code that works on the subkey generation. 

Because of this, to problem solve around this issue is: 

1. Go through the key generation algorithm and make arrangements to print all the rounds subkeys. 
2. Once subkeys are ensured, try the encryption algorithm and print encryption at all rounds and its subkeys. 
3. Once the encryption for this assignment works, reverse engineer for decryption. 
4. After encrypting and decrypting hexadecimal string, improve the project by being able to convert and encrypt using plain text message such as "Hello, World" as input instead of hexadecimal numbers.  
